# Tut4 - Automation. Using Access Token Authentication and Updating Domo via API

- In this tutorial we will modify all our code to use access_token authentication.
- We will learn to create, search for, retrieve and delete access_tokens via API
- We will create Account objects in Domo

## 🎓 What is Automation or Integration?

> "I want to generate and store my access_token in Domo Accounts"

Writing Integrations = decomposing a multiple step process into smaller pieces.

1. Use the UI to write down how you might approach this task by hand.
2. Each mouse-click often corresponds to an API request. Stub out some functions to capture the activities you're taking
3. Monitor network traffic to confirm assumptions.

### Possible Solution
Break into teams.

## ⭐ Team 1. Design a workflow that can regenerate or create an access_token given a user_id and token_name

## ⭐ Team 2. Design a workflow that can create or update a Domo Account (access_token and abstract credentials store).

- note updating the account is two steps, updating metadata vs. updating the actual configuration

## Team 3. Modify format_domostats_account to include config information from accounts (that share the same name) with the jupyter workspace

- in our code we will match on account name.  what are the risks?


## ⭐ Team 4. MOST DIFFICULT - Design a process to automatically share existing accounts with the DomoJupyter workspace then 

use DomoJupyter routes as a reference
https://github.com/jaewilson07/domolibrary/blob/main/domolibrary/routes/jupyter.py
https://github.com/jaewilson07/domolibrary/blob/main/nbs/routes/jupyter.ipynb


### Utils


In [ ]:
import datetime as dt


def convert_domo_utc_to_datetime(timestamp):
    return dt.datetime.fromtimestamp(timestamp / 1000, dt.timezone.utc)

In [ ]:
# Domo returns token expiration date in unix time

convert_domo_utc_to_datetime(1904914053000)

In [ ]:
import time


def generate_domo_expiration_unixtimestamp(
    duration_in_days: int = 15, debug_prn: bool = False
):
    """# the expiration date of the access token is calculated based off of x days into the future which must then be converted into a unix timestamp"""

    today = dt.datetime.today()
    expiration_date = today + dt.timedelta(days=duration_in_days)

    if debug_prn:
        print(f"expiration_date is {duration_in_days} from today {expiration_date}")

    return int(time.mktime(expiration_date.timetuple()) * 1000)

In [ ]:
# the expiration date of the access token is calculated based off of x days into the future which must then be converted into a unix timestamp

generate_domo_expiration_unixtimestamp(duration_in_days=15)

## Auth

In this project, we use `access_token` (retrieved from an "access_token" account shared with DomoJupyter) to authenticate API requests.  

Note that access_token auth can store everything except the Domo Instance.

In [ ]:
from solutions.utils import read_domo_jupyter_account

account_name = 'your_account_name'
creds = read_domo_jupyter_account(account_name)

access_token = creds['domoAccessToken']
domo_username = creds['username']
domo_password = creds['password']

domo_instance = 'domo-community'

## Solution Code starts here

### Access Tokens


In [ ]:
# from  solutions.access_tokens import get_all_access_tokens

import requests
from solutions.client import ResponseClass
from solutions.utils import convert_domo_utc_to_datetime


def get_all_access_tokens(
    domo_instance,
    session_token=None,
    access_token = None,
    headers=None,
    return_raw: bool = False,
    debug_api: bool = False,
):

    url = f"https://{domo_instance}.domo.com/api/data/v1/accesstokens"

    headers = headers or {}

    if session_token:
        headers.update({"x-domo-authentication": session_token})
        
    if access_token:
        headers.update({"x-domo-developer-token": access_token})

    if debug_api:
        print({"url": url, "headers": headers})

    res = requests.request(
        url=url,
        method="GET",
        headers=headers,
    )

    res = ResponseClass.from_request_response(res)

    if return_raw:
        return res

    [
        token.update({"expires": convert_domo_utc_to_datetime(token["expires"])})
        for token in res.response
    ]

    return res

In [ ]:

# note - because this has "functionality" beyond the base API
# you might argue that could be implemented as a class method of a "class_function" 
# as described in the Extra Credit portion of Tutorial 1.

def search_tokens_by_user_id_and_token_name(
    domo_instance, 
    user_id,
    token_name,
    session_token = None,
    access_token = None,
):
    res = get_all_access_tokens(
        session_token=session_token,
        access_token = access_token,
        domo_instance=domo_instance,
        return_raw=True,
        debug_api=False,
    )

    all_tokens = res.response

    return next(
        (
            token
            for token in all_tokens
            if token["name"] == token_name and token["ownerId"] == user_id
        ),
        None,
    )


test_token = search_tokens_by_user_id_and_token_name(
    access_token =access_token,
    domo_instance=domo_instance,
    user_id=85654293,
    token_name="dp24",
)

test_token

In [ ]:
import pandas as pd

res = get_all_access_tokens(
    # session_token=test_session_token,
    access_token = access_token,
    domo_instance=domo_instance,
    return_raw=True,
    debug_api=False,
)
test_access_tokens = res.response
test_access_tokens[0:5]
# pd.DataFrame(test_access_tokens)[0:5]

# Users


In [ ]:
from solutions.users import get_user_by_id

def get_user_by_id(
    domo_instance,
    user_id: int,
    session_token=None,
    access_token = None,
    headers=None,
    return_raw: bool = False,
    debug_api: bool = False,
):
    url = f"https://{domo_instance}.domo.com/api/content/v2/users/{user_id}"

    headers = headers or {}

    if session_token:
        headers.update({"x-domo-authentication": session_token})
        
    if access_token:
        headers.update({"x-domo-developer-token": access_token})

    if debug_api:
        print({"url": url, "headers": headers})
        
    res = requests.request(
        url=url,
        method="GET",
        headers=headers,
    )

    res = ResponseClass.from_request_response(res)

    if return_raw:
        return res

    return res

In [ ]:
get_user_by_id(
    domo_instance=domo_instance,
    access_token=access_token,
    user_id=85654293,
    debug_api=False,
)

In [ ]:
from solutions.access_tokens import generate_access_token

from pprint import pprint


def generate_access_token(
    domo_instance,
    token_name: str,
    user_id,
    duration_in_days: 15,
    session_token=None,
    access_token = None,
    headers=None,
    return_raw: bool = False,
    debug_api: bool = False,
):
    url = f"https://{domo_instance}.domo.com/api/data/v1/accesstokens"

    expiration_timestamp = generate_domo_expiration_unixtimestamp(
        duration_in_days=duration_in_days
    )

    body = {"name": token_name, "ownerId": user_id, "expires": expiration_timestamp}

    headers = headers or {}

    if session_token:
        headers.update({"x-domo-authentication": session_token})
        
    if access_token:
        headers.update({"x-domo-developer-token": access_token})

    if debug_api:
        print({"url": url, "headers": headers, "body" : body})
        
    res = requests.request(method="post", url=url, json=body, headers=headers)

    res = ResponseClass.from_request_response(res)
    if return_raw:
        return res

    return res

In [ ]:
generate_access_token(
    domo_instance=domo_instance,
    access_token=access_token,
    token_name="dp24",
    duration_in_days=15,
    user_id=85654293,
    debug_api=False,
)

In [ ]:
from solutions.access_tokens import revoke_access_token

def revoke_access_token(
    domo_instance,
    access_token_id,
    access_token = None,
    session_token=None,
    headers=None,
    return_raw: bool = False,
    debug_api: bool = False,
):

    url = f"https://{domo_instance}.domo.com/api/data/v1/accesstokens/{access_token_id}"

    headers = headers or {}

    if session_token:
        headers.update({"x-domo-authentication": session_token})
        
    if access_token:
        headers.update({"x-domo-developer-token": access_token})

    if debug_api:
        print({"url": url, "headers": headers})
    res = requests.request(method="DELETE", url=url, headers=headers)

    res = ResponseClass.from_request_response(res)
    if return_raw:
        return res

    res.response = f"access token {access_token_id} deleted"
    return res


## CUSTOM WORKFLOW FOR TESTING CODE
# so that we don't generate a ton of the same access token
# create access token
# find access token
# immediately delete it

generate_access_token(
    domo_instance=domo_instance,
    access_token=access_token,
    token_name="dp24",
    duration_in_days=15,
    user_id=85654293,
    debug_api=False,
)

test_token = search_tokens_by_user_id_and_token_name(
    access_token=access_token,
    domo_instance=domo_instance,
    user_id=85654293,
    token_name="dp24",
)

if not test_token:
    raise Exception("no token")

res =revoke_access_token(domo_instance = domo_instance,
                    access_token=access_token,
                    access_token_id=test_token['id'],
                    return_raw=False,
                    debug_api=False)

test_token

### Domo Accounts


In [ ]:
from solutions.accounts import get_accounts

def get_accounts(domo_instance, 
                 session_token = None,
                 access_token = None,
                 headers : dict = None,
                 debug_api: bool = False,
                 return_raw: bool = False):

    url = f"https://{domo_instance}.domo.com/api/data/v1/accounts"

    headers = headers or {}

    if session_token:
        headers.update({"x-domo-authentication": session_token})
        
    if access_token:
        headers.update({"x-domo-developer-token": access_token})

    if debug_api:
        print({"url": url, "headers": headers})
        
    res = requests.request(method="GET", url=url, headers=headers)

    res = ResponseClass.from_request_response(res)

    return res

In [ ]:
res = get_accounts(
    domo_instance=domo_instance,
    access_token=access_token
)
pd.DataFrame(res.response[0:5])

In [ ]:
from solutions.accounts import generate_account__abstract_credential_store_body
import datetime as dt
import json


def generate_account__abstract_credential_store_body(
    account_name, credentials, is_timestamp: bool = True
):

    # the API expects credentials to be type string, so conditionally convert dict to string
    if isinstance(credentials, dict):
        credentials = json.dumps(credentials)

    if is_timestamp:
        account_name = f"{account_name} - updated {dt.date.today()}"
    return {
        "name": "Abstract Credential Store Account",
        "displayName": account_name,
        "dataProviderType": "abstract-credential-store",
        "configurations": {"credentials": credentials},
    }


generate_account__abstract_credential_store_body(
    "test", "my creds go here", is_timestamp=True
)

In [ ]:
from solutions.accounts import generate_account__access_token_body

def generate_account__access_token_body(
    account_name, access_token=None, username=None, password=None
):
    return {
        "name": "Domo Access Token Account",
        "displayName": account_name,
        "dataProviderType": "domo-access-token",
        "configurations": {
            "domoAccessToken": {access_token},
            "username": {username},
            "password": {password},
        },
    }


generate_account__access_token_body(
    account_name="jw_dp24", access_token="123", username="jae@test.com", password=123
)

In [ ]:
from solutions.accounts import create_account 

import requests


def create_account(body,
                   domo_instance,
                   headers = None,
                   session_token = None,
                   access_token = None,
                   debug_api: bool = False):
    
    url = f"https://{domo_instance}.domo.com/api/data/v1/accounts"

    headers = headers or {}

    if session_token:
        headers.update({"x-domo-authentication": session_token})
        
    if access_token:
        headers.update({"x-domo-developer-token": access_token})

    if debug_api:
        print({"url": url, "headers": headers})

    res = requests.request(method="POST", url=url, json=body, headers=headers)

    res = ResponseClass.from_request_response(res)

    if not res.is_success:
        raise DomoAPIRequest_Error(create_account.__name__, res)

    return res


create_account_body = generate_account__abstract_credential_store_body(
    account_name="my_domo_community_access_token", credentials=test_token
)

create_account(
    body=create_account_body,
    domo_instance=domo_instance,
    access_token=access_token,
)

In [ ]:
from solutions.accounts import get_account_by_id

def get_account_by_id(
    domo_instance,
    account_id,
    headers = None,
    session_token=None,
    access_token=None,
    return_raw: bool = False,
    debug_api: bool = False,
):
    url = f"https://{domo_instance}.domo.com/api/data/v1/accounts/{account_id}?unmask=true"

    headers = headers or {}

    if session_token:
        headers.update({"x-domo-authentication": session_token})
        
    if access_token:
        headers.update({"x-domo-developer-token": access_token})

    if debug_api:
        print({"url": url, "headers": headers})

    res = requests.request(method="GET", url=url, headers=headers)

    res = ResponseClass.from_request_response(res)

    if return_raw:
        return res

    return res

In [ ]:
get_account_by_id(
    domo_instance=DOMO_INSTANCE,
    account_id=102,
    session_token=test_session_token,
    debug_api=True,
)

In [ ]:
from solutions.accounts import update_account_config

def update_account_config(
    domo_instance,
    dataprovider_type,
    account_id,
    body: dict,  # only receives configuration portion of the account definition
    headers: dict = None,
    session_token: str = None,
    access_token: str = None,
    debug_api: bool = False,
    return_raw: bool = False,
):
    url = f"https://{domo_instance}.domo.com/api/data/v1/providers/{dataprovider_type}/account/{account_id}"

    headers = headers or {}

    if session_token:
        headers.update({"x-domo-authentication": session_token})
        
    if access_token:
        headers.update({"x-domo-developer-token": access_token})

    if debug_api:
        print({"url": url, "headers": headers})
    res = requests.request(method="PUT", json=body, url=url, headers=headers)

    res = ResponseClass.from_request_response(res)

    if return_raw:
        return res

    return res

In [ ]:
create_account_body = generate_account__abstract_credential_store_body(
    account_name=f"my_domo_community_access_token - {dt.datetime.now()}",
    credentials=test_token,
)

# NOTE
config = create_account_body["configurations"]

update_account_config(
    domo_instance=domo_instance,
    account_id=102,
    dataprovider_type="abstract-credential-store",
    body=config,
    access_token=access_token,
    debug_api=True,
)

In [ ]:
from solutions.accounts import update_account_name

# notice that we pass a string to requests instead of a json object!!

def update_account_name(
    domo_instance: str,
    account_id: int,
    account_name: str,
    session_token=None,
    access_token= None,
    headers: dict = None,
    debug_api: bool = False,
    return_raw: bool = False,
):
    url = f"https://{domo_instance}.domo.com/api/data/v1/accounts/{account_id}/name"

    headers = headers or {}

    if session_token:
        headers.update({"x-domo-authentication": session_token})
        
    if access_token:
        headers.update({"x-domo-developer-token": access_token})

    if debug_api:
        print({"url": url, "headers": headers, "body": account_name})

    res = requests.request(method="PUT", data=account_name, url=url, headers=headers)

    res = ResponseClass.from_request_response(res)

    if return_raw:
        return res

    return res

In [ ]:
update_account_name(
    account_id=102,
    account_name=f"dp24 - update {dt.datetime.now()}",
    domo_instance=domo_instance,
    access_token=access_token,
    debug_api=True,
)